In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image
import pandas as pd
import random
import shutil
import cv2

In [93]:
class_names = ["cleaned", "dirty"]
train_path = "/Users/sergeiakhmadulin/Documents/No_background/train/"
val_path = "/Users/sergeiakhmadulin/Documents/No_background/valid/"
test_path = "/Users/sergeiakhmadulin/Documents/No_background/test/"

In [94]:
for class_name in class_names:
    full_path = train_path + class_name + "/"
    print(f"Files in train folder of class {class_name}: {len(list(filter(lambda x: x.endswith('.jpg'), os.listdir(full_path))))}")

Files in train folder of class cleaned: 20
Files in train folder of class dirty: 20


In [5]:
class MyRemoveBackground:
    """Remove images background.
    """
    
    def __init__(self):
        pass

    def __call__(self, in_img):
        
        # Convert PIL image to numpy array
        in_img = np.array(in_img)
        
        # Get the height and width from OpenCV image
        height, width = in_img.shape[:2]
        
        # Create a mask holder
        mask = np.zeros([height, width], np.uint8)

        # Grab Cut the object
        bgdModel = np.zeros((1, 65),np.float64)
        fgdModel = np.zeros((1, 65),np.float64)

        # Hard Coding the Rect The object must lie within this rect.
        rect = (15, 15, width-30, height-30)
        cv2.grabCut(in_img, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
        mask = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
        out_img = in_img * mask[:, :, np.newaxis]

        # Get the background
        background = in_img - out_img

        # Change all pixels in the background that are not black to white
        background[np.where((background > [0, 0, 0]).all(axis = 2))] = [255, 255, 255]

        #Add the background and the image
        out_img = background + out_img

        return Image.fromarray(out_img)

In [6]:
def remove_background(image_roots):
    """Remove picture background.
       This function use MyRemoveBackground class.
    """
    remove_photo_background = MyRemoveBackground()

    print('Backgrounds removing started...')
    for path in image_roots:
        files = os.listdir(path)
        files = list(filter(lambda x: x.endswith('.jpg'), files))
        
        print(f'{len(files)} pictures was found in {path}', end='')
        for i, file in enumerate(files):
            img_original = cv2.imread(path + file)
            img_cleaned = remove_photo_background(img_original)
            img_cleaned = np.array(img_cleaned)
            cv2.imwrite(path + file, img_cleaned)
            if i % 20 == 0:
                print('\n{:>3d}/{:>3d}'.format(i, len(files)), end='')
            print('.', end='')
        print()
    print('Backgrounds removing is complete.\n')

In [95]:
remove_background(image_roots=[os.path.join(train_path, 'cleaned/'),
                               os.path.join(train_path, 'dirty/')
#                                ,os.path.join(data_root, 'test/')
                              ])

Backgrounds removing started...
20 pictures was found in /Users/sergeiakhmadulin/Documents/No_background/train/cleaned/
  0/ 20....................
20 pictures was found in /Users/sergeiakhmadulin/Documents/No_background/train/dirty/
  0/ 20....................
Backgrounds removing is complete.



In [196]:
def move_random_files(source_folder, destination_folder, ratio_for_val):
    # Get all files in the source folder
    for class_name in class_names:
        full_path_source = source_folder + class_name + "/"
        full_destination_folder = destination_folder + class_name + "/"
        os.makedirs(full_destination_folder, exist_ok=True)
        files = os.listdir(full_path_source)
        files = list(filter(lambda x: x.endswith('.jpg'), files))
        
        whole_part = int(len(files) * ratio_for_val)
        # Randomly select the specified number of files
        if len(files) < whole_part:
            print(f"Not enough files to choose from. Available: {len(files)}")
            return
    
        selected_files = random.sample(files, whole_part)
        # Move each selected file to the destination folder
        for file_name in selected_files:
            shutil.move(os.path.join(full_path_source, file_name), os.path.join(full_destination_folder, file_name))
    print(f'Moved to validation folder: {len(selected_files)} files of each class')

In [97]:
move_random_files(train_path, val_path, 0.2)

/Users/sergeiakhmadulin/Documents/No_background/valid/cleaned/
Moved to validation folder: 0011.jpg
Moved to validation folder: 0017.jpg
Moved to validation folder: 0004.jpg
Moved to validation folder: 0015.jpg
/Users/sergeiakhmadulin/Documents/No_background/valid/dirty/
Moved to validation folder: 0005.jpg
Moved to validation folder: 0018.jpg
Moved to validation folder: 0003.jpg
Moved to validation folder: 0012.jpg


In [98]:
target_size = 224

In [99]:
def add_gaussian_noise(image):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1, dtype=tf.float32)
    noisy_image = image + noise
    return noisy_image


def preprocess_image(image):
    # Randomly choose a transformation
    transform_choice = np.random.choice(['none', 'noise', 'grayscale', 'brightness'])
    if transform_choice == 'noise':
        image = add_gaussian_noise(image)
    elif transform_choice == 'grayscale':
        image = tf.image.rgb_to_grayscale(image)
    elif transform_choice == 'brightness':
        image = tf.image.random_brightness(image, max_delta=0.2)

    return image

In [100]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,    
    rotation_range=90,
    vertical_flip=True,
    channel_shift_range = 0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [101]:
def generate_augm_data(datagen_function, folder, num_of_img, classes):
    print("Augmantation started.")
    for class_name in classes:
        full_path = folder + class_name + "/"
        original_images = []  # List to hold your original images
        files = os.listdir(full_path)
        files = list(filter(lambda x: x.endswith('.jpg'), files))
        for num_img, filename in enumerate(files):
            img = Image.open(os.path.join(full_path, filename))
            original_images.append(np.array(img))
        num_of_original_images = len(original_images)
        print(f"\nFound original images of class {class_name}: {num_of_original_images}")
        num_of_each_img = num_of_img//num_of_original_images
        num_of_new = 0
        for num_orig, img in enumerate(original_images):
            img = img.reshape((1,) + img.shape)  # Reshape for the generator
            current_count = 0
            for num_augm, batch in enumerate(datagen.flow(img, batch_size=1)):
                augmented_image = batch[0].astype(np.uint8)
                # Save or process your augmented images
                Image.fromarray(augmented_image).save(full_path + f'augmented_image_{num_orig}_{num_augm}.jpg')
                current_count += 1
                num_of_new += 1
                if current_count >= num_of_each_img:
                    break
        print(f"Add new augmented images of class {class_name}: {num_of_new}")
    print("\nAugmentation finished.")

In [102]:
generate_augm_data(datagen, train_path, 30000, class_names)

Augmantation started.

Found original images of class cleaned: 16
Add new augmented images of class cleaned: 30000

Found original images of class dirty: 16
Add new augmented images of class dirty: 30000

Augmentation finished.


In [105]:
generate_augm_data(datagen, val_path, 700, class_names)

Augmantation started.

Found original images of class cleaned: 4
Add new augmented images of class cleaned: 700

Found original images of class dirty: 4
Add new augmented images of class dirty: 700

Augmentation finished.


In [103]:
def train_val_datasets():
    """Creates datasets for training and validation.

    Returns:
        (tf.data.Dataset, tf.data.Dataset): Training and validation datasets.
    """

    training_dataset = tf.keras.utils.image_dataset_from_directory( 
        directory=train_path,
        batch_size=16,
        image_size=(224, 224),
        shuffle=True, 
    ) 
    
    validation_dataset = tf.keras.utils.image_dataset_from_directory( 
        directory=val_path,
        batch_size=16,
        image_size=(224, 224),
        shuffle=True, 
    ) 

    return training_dataset, validation_dataset

In [106]:
training_dataset, validation_dataset = train_val_datasets()

Found 60032 files belonging to 2 classes.
Found 1408 files belonging to 2 classes.


In [59]:
LOCAL_WEIGHTS_FILE = '/Users/sergeiakhmadulin/My Drive/Clean-Dirty/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [273]:
def create_pre_trained_model():
    """Creates the pretrained inception V3 model

    Returns:
        tf.keras.Model: pre-trained model
    """

    ### START CODE HERE ###
    
    pre_trained_model = tf.keras.applications.EfficientNetB0( 
        include_top=False, 
        input_shape=(224, 224, 3),
        weights='imagenet'
    ) 

    # Make all the layers in the pre-trained model non-trainable
    # pre_trained_model.load_weights(LOCAL_WEIGHTS_FILE)
    pre_trained_model.trainable = False

    ### END CODE HERE ###

    return pre_trained_model

In [283]:
base_model = create_pre_trained_model()

In [284]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

In [285]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,055,972 (15.47 MB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [250]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.99 and logs['val_accuracy'] >= 0.99:
            self.model.stop_training = True

            print("\nReached 80% train accuracy and 80% validation accuracy, so cancelling training!")


checkpoint = ModelCheckpoint(
    'best_model.keras',               # File name to save the model
    monitor='val_accuracy',         # Metric to monitor
    save_best_only=True,           # Save only the best model
    mode='max',                    # Mode (max for accuracy)
    verbose=1                      # Verbosity
)

In [237]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
history = model.fit(
    training_dataset,
    validation_data = validation_dataset,
    epochs = 10,
    verbose = 1,
    callbacks = [EarlyStoppingCallback()],
)

Epoch 1/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 566s 150ms/step - accuracy: 0.8654 - loss: 0.3043 - val_accuracy: 0.7038 - val_loss: 1.0416
Epoch 2/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 565s 151ms/step - accuracy: 0.9694 - loss: 0.0886 - val_accuracy: 0.7109 - val_loss: 1.2013
Epoch 3/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 560s 149ms/step - accuracy: 0.9788 - loss: 0.0638 - val_accuracy: 0.7131 - val_loss: 1.3088
Epoch 4/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 565s 151ms/step - accuracy: 0.9802 - loss: 0.0567 - val_accuracy: 0.7124 - val_loss: 1.3807
Epoch 5/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 550s 147ms/step - accuracy: 0.9819 - loss: 0.0515 - val_accuracy: 0.7173 - val_loss: 1.4168
Epoch 6/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 594s 158ms/step - accuracy: 0.9827 - loss: 0.0508 - val_accuracy: 0.7195 - val_loss: 1.4037
Epoch 7/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 596s 159ms/step - accuracy: 0.9829 - loss: 0.0496 - val_accuracy: 0.7195 - val_loss: 1.4524
Epoch 8/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 601s 160ms/step - ac

In [281]:
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze first 100 layers
    layer.trainable = False

# Compile and train the model again
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [282]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,055,972 (15.47 MB)

 Trainable params: 2,054,545 (7.84 MB)

 Non-trainable params: 2,001,427 (7.63 MB)

In [115]:
history = model.fit(
    training_dataset,
    validation_data = validation_dataset,
    epochs = 10,
    verbose = 1,
    callbacks = [EarlyStoppingCallback()],
)

Epoch 1/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1296s 344ms/step - accuracy: 0.9750 - loss: 0.0662 - val_accuracy: 0.7202 - val_loss: 1.7308
Epoch 2/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1197s 319ms/step - accuracy: 0.9938 - loss: 0.0167 - val_accuracy: 0.7379 - val_loss: 1.9163
Epoch 3/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1139s 304ms/step - accuracy: 0.9968 - loss: 0.0093 - val_accuracy: 0.8310 - val_loss: 1.3866
Epoch 4/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1106s 295ms/step - accuracy: 0.9970 - loss: 0.0075 - val_accuracy: 0.7266 - val_loss: 1.4376
Epoch 5/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1110s 296ms/step - accuracy: 0.9969 - loss: 0.0078 - val_accuracy: 0.8054 - val_loss: 1.6076
Epoch 6/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1152s 307ms/step - accuracy: 0.9979 - loss: 0.0052 - val_accuracy: 0.7614 - val_loss: 1.5370
Epoch 7/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1180s 314ms/step - accuracy: 0.9975 - loss: 0.0062 - val_accuracy: 0.7564 - val_loss: 1.4362
Epoch 8/10
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 1160s 309ms/s

In [116]:
def preprocess_image(image_path, target_size):
    remove_photo_background = MyRemoveBackground()
    img = load_img(image_path, target_size=(224,224))  # Resize to match model's input shape
    img_array = remove_photo_background(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [147]:
results = []
files = list(filter(lambda x: x.endswith('.jpg'), os.listdir(test_path)))
for filename in files:
    image_path = os.path.join(test_path, filename)
    processed_image = preprocess_image(image_path, target_size)

    # Predict the label
    prediction = model.predict(processed_image, verbose=0)
    
    # Get the predicted class (for example, using argmax for multi-class classification)
    predicted_class = np.argmax(prediction, axis=1)
    results.append({'id': filename, 'label': predicted_class, 'probs': prediction})

In [126]:
results_df_test = pd.DataFrame(results)

In [127]:
results_df_test["dirty"] = np.where(results_df_test["probs"]>=0.99,1,0)
results_df_test["cleaned"] = np.where(results_df_test["probs"]<=0.01,1,0)

In [152]:
df_cleaned = results_df_test[results_df_test["cleaned"]==1]
id_cleaned = list(df_cleaned["id"])

df_dirty = results_df_test[results_df_test["dirty"]==1]
id_dirty = list(df_dirty["id"])

min_value = min(len(id_cleaned), len(id_dirty))

index_to_save = random.sample(range(min_value), min_value)

files_clean_to_stay = list(df_cleaned.iloc[index_to_save]["id"])
files_dirty_to_stay = list(df_dirty.iloc[index_to_save]["id"])

In [206]:
def make_new_dir_with_files_to_train(source_folder, destination_folder, 
                                     list_given_file_names_cleaned, list_given_file_names_dirty):

    # Get all files in the source folder
    for class_name in class_names:
        full_path_source = source_folder + "/"
        full_destination_folder = destination_folder + class_name + "/"
        os.makedirs(full_destination_folder, exist_ok=True)
        
        if class_name == "cleaned":
            matched_files_cleaned = [file for file in os.listdir(full_path_source) if file in list_given_file_names_cleaned]
            for file_name in matched_files_cleaned:
                shutil.copy(os.path.join(full_path_source, file_name), os.path.join(full_destination_folder, file_name))

        else:
            matched_files = [file for file in os.listdir(full_path_source) if file in list_given_file_names_dirty]
            for file_name in matched_files:
                shutil.copy(os.path.join(full_path_source, file_name), os.path.join(full_destination_folder, file_name))
            
    print(f'Copied files from test to new folder to train: {len(list_given_file_names_dirty)}')

In [203]:
path_new_files_train = "/Users/sergeiakhmadulin/Documents/new_files/train/"
path_new_files_val = "/Users/sergeiakhmadulin/Documents/new_files/valid/"

In [207]:
make_new_dir_with_files_to_train(test_path, path_new_files_train, 
                                     files_clean_to_stay, files_dirty_to_stay)

Copied files from test to new folder to train: 70


In [208]:
remove_background(image_roots=[os.path.join(path_new_files_train, 'cleaned/'),
                               os.path.join(path_new_files_train, 'dirty/')
                              ])

Backgrounds removing started...
70 pictures was found in /Users/sergeiakhmadulin/Documents/new_files/train/cleaned/
  0/ 70....................
 20/ 70....................
 40/ 70....................
 60/ 70..........
70 pictures was found in /Users/sergeiakhmadulin/Documents/new_files/train/dirty/
  0/ 70....................
 20/ 70....................
 40/ 70....................
 60/ 70..........
Backgrounds removing is complete.



In [209]:
move_random_files(path_new_files_train, path_new_files_val, 0.2)

Moved to validation folder: 14 files of each class


In [210]:
generate_augm_data(datagen, path_new_files_train, 30000, class_names)

Augmantation started.

Found original images of class cleaned: 56
Add new augmented images of class cleaned: 29960

Found original images of class dirty: 56
Add new augmented images of class dirty: 29960

Augmentation finished.


In [211]:
generate_augm_data(datagen, path_new_files_val, 700, class_names)

Augmantation started.

Found original images of class cleaned: 14
Add new augmented images of class cleaned: 700

Found original images of class dirty: 14
Add new augmented images of class dirty: 700

Augmentation finished.


In [222]:
def rename_files_in_directory(directory, new_prefix):
    # Loop through all files in the specified directory
    for index, filename in enumerate(os.listdir(directory)):
        # Create the full file path
        old_file_path = os.path.join(directory, filename)
        
        # Check if it is a file (not a directory)
        if os.path.isfile(old_file_path):
            # Create new file name with the specified prefix
            new_filename = f"{new_prefix}_{index + 1}{os.path.splitext(filename)[1]}"
            new_file_path = os.path.join(directory, new_filename)
            
            # Rename the file
            os.rename(old_file_path, new_file_path)

In [225]:
rename_files_in_directory(path_new_files_train + "cleaned" + "/", "_new")
rename_files_in_directory(path_new_files_train + "dirty" + "/", "_new")

In [231]:
rename_files_in_directory(path_new_files_val + "cleaned" + "/", "_new")
rename_files_in_directory(path_new_files_val + "dirty" + "/", "_new")

In [229]:
def copy_files(path_from, path_to, class_names):
    for class_name in class_names:
        full_sourse_folder = path_from + class_name + "/"
        full_destination_folder = path_to + class_name + "/"
        files_to_copy = list(filter(lambda x: x.endswith('.jpg'), os.listdir(full_sourse_folder)))
        print(full_sourse_folder, full_destination_folder)
        for filename in files_to_copy:
            shutil.copy(os.path.join(full_sourse_folder, filename), full_destination_folder)
        files_in_folder = len(list(filter(lambda x: x.endswith('.jpg'), os.listdir(full_destination_folder))))
        print(f"Copy fineshed. In folder {full_destination_folder} now: {files_in_folder} files")

In [230]:
copy_files(path_new_files_train, train_path, class_names)

/Users/sergeiakhmadulin/Documents/new_files/train/cleaned/ /Users/sergeiakhmadulin/Documents/No_background/train/cleaned/
Copy fineshed. In folder /Users/sergeiakhmadulin/Documents/No_background/train/cleaned/ now: 30016 files
/Users/sergeiakhmadulin/Documents/new_files/train/dirty/ /Users/sergeiakhmadulin/Documents/No_background/train/dirty/
Copy fineshed. In folder /Users/sergeiakhmadulin/Documents/No_background/train/dirty/ now: 30016 files


In [232]:
copy_files(path_new_files_val, train_path, class_names)

/Users/sergeiakhmadulin/Documents/new_files/valid/cleaned/ /Users/sergeiakhmadulin/Documents/No_background/train/cleaned/
Copy fineshed. In folder /Users/sergeiakhmadulin/Documents/No_background/train/cleaned/ now: 714 files
/Users/sergeiakhmadulin/Documents/new_files/valid/dirty/ /Users/sergeiakhmadulin/Documents/No_background/train/dirty/
Copy fineshed. In folder /Users/sergeiakhmadulin/Documents/No_background/train/dirty/ now: 714 files


In [238]:
training_dataset, validation_dataset = train_val_datasets()

Found 120072 files belonging to 2 classes.
Found 2836 files belonging to 2 classes.


In [239]:
history_extended_data_base_model = model.fit(
    training_dataset,
    validation_data = validation_dataset,
    epochs = 10,
    verbose = 1,
    callbacks = [EarlyStoppingCallback()],
)

Epoch 1/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 940s 125ms/step - accuracy: 0.9051 - loss: 0.2353 - val_accuracy: 0.8889 - val_loss: 0.2794
Epoch 2/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 913s 122ms/step - accuracy: 0.9661 - loss: 0.0971 - val_accuracy: 0.8963 - val_loss: 0.2759
Epoch 3/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 922s 123ms/step - accuracy: 0.9734 - loss: 0.0782 - val_accuracy: 0.9027 - val_loss: 0.2596
Epoch 4/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 918s 122ms/step - accuracy: 0.9747 - loss: 0.0722 - val_accuracy: 0.9030 - val_loss: 0.2827
Epoch 5/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 921s 123ms/step - accuracy: 0.9758 - loss: 0.0692 - val_accuracy: 0.9030 - val_loss: 0.2856
Epoch 6/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 923s 123ms/step - accuracy: 0.9767 - loss: 0.0646 - val_accuracy: 0.9044 - val_loss: 0.2840
Epoch 7/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 925s 123ms/step - accuracy: 0.9779 - loss: 0.0638 - val_accuracy: 0.9066 - val_loss: 0.2770
Epoch 8/10
7505/7505 ━━━━━━━━━━━━━━━━━━━━ 926s 123ms/step - ac

In [286]:
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze first 100 layers
    layer.trainable = False

# Compile and train the model again
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [287]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,055,972 (15.47 MB)

 Trainable params: 3,844,189 (14.66 MB)

 Non-trainable params: 211,783 (827.28 KB)

In [ ]:
history_extended_data_model_tr = model.fit(
    training_dataset,
    validation_data = validation_dataset,
    epochs = 10,
    verbose = 1,
    callbacks = [EarlyStoppingCallback(), checkpoint],
)

Epoch 1/10
2591/7505 ━━━━━━━━━━━━━━━━━━━━ 24:08 295ms/step - accuracy: 0.9570 - loss: 0.1037

In [253]:
model.save_weights('/Users/sergeiakhmadulin/Documents/Cleaned/model_weights.weights.h5')

In [122]:
results_df_test["label"] = np.where(results_df_test["probs"]>=0.5,"dirty","cleaned")

In [123]:
results_df_test = results_df_test.iloc[:,:2]
results_df_test['id'] = results_df_test['id'].apply(lambda x: x.split('.')[0])

In [124]:
results_df_test

,id,label
0,0071,cleaned
1,0717,dirty
2,0703,dirty
3,0065,dirty
4,0059,dirty
...,...,...
739,0040,dirty
740,0726,cleaned
741,0732,dirty
742,0054,dirty


In [125]:
results_df_test.to_csv('/Users/sergeiakhmadulin/Documents/Cleaned/submission_1.csv',index=False)